In [2]:
# Importa las bibliotecas necesarias
from river.drift import ADWIN
from river.ensemble import HoeffdingTreeClassifier
from river import datasets
from river import metrics
from river import anomaly

# Crea instancias de los algoritmos y métricas
adwin = ADWIN()
ensemble = HoeffdingTreeClassifier()
metric_acc = metrics.Accuracy()
metric_f1 = metrics.F1Score()

# Inicializa el detector de outliers
isolation_forest = anomaly.IsolationForest()

# Carga el conjunto de datos
data = datasets.CreditCard()

# Inicializa variables para el seguimiento
n_samples = 0
change_detected = False

# Divide el conjunto de datos en entrenamiento y tiempo real
train_data, real_time_data = data.split(n=50000)

# Realiza el entrenamiento inicial
for x, y in train_data:
    y_pred = ensemble.predict_one(x)
    ensemble.learn_one(x, y)
    metric_acc.update(y, y_pred)
    metric_f1.update(y, y_pred)
    n_samples += 1

# Itera a través de los datos en tiempo real
for x, y in real_time_data:
    n_samples += 1

    # Realiza la detección de cambios
    if adwin.update(y):
        change_detected = True
        print(f"Change detected at sample {n_samples}")

    # Realiza la detección de outliers
    is_outlier = isolation_forest.score_one(x) < 0.5

    # Actualiza el clasificador de ensamble con los nuevos datos
    y_pred = ensemble.predict_one(x)
    ensemble.learn_one(x, y)

    # Evalúa el rendimiento
    metric_acc.update(y, y_pred)
    metric_f1.update(y, y_pred)

    # Si es un outlier, puedes tomar medidas específicas, como notificar o registrar el evento
    if is_outlier:
        print(f"Outlier detected at sample {n_samples}")

# Muestra las métricas finales
print(f"Number of samples: {n_samples}")
print("Accuracy:", metric_acc.get())
print("F1 Score:", metric_f1.get())


ModuleNotFoundError: No module named 'google'

: 